# Issue53

* Timelink Python package version: 1.1.13
* Python version: 3.10, 3.11
* Operating System: MacOs

### Description

it is not always possible to get the ORM corresponding to a group because a group
can extend another one and not have its own mapping. However internally Timelink
knows the model to use, but the end user need to inspect the structure file to figure out.

In str
```
pars name=lugar; source=geoentity
```

in python
```python
db.get_model("lugar") # returns none
```

Or make get_model smarter.

In [1]:
# NBVAL_IGNORE_OUTPUT
from timelink.notebooks import TimelinkNotebook
tlnb = TimelinkNotebook(
    kleio_image='kleio-server',
    kleio_version='12.6.575',
    db_name='issue53',
    db_type='sqlite',)

tlnb.print_info()

Timelink version: 1.1.13
Project name: test-project
Project home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project
Database type: sqlite
Database name: issue53
Kleio image: kleio-server
Kleio server token: t3pXK...
Kleio server URL: http://127.0.0.1:8088
Kleio server home: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project
Kleio server container: youthful_mahavira
Kleio version requested: 12.6.575
Kleio server version: 12.6.575 (2024-09-06 14:08:58)
SQLite directory: /Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project/database/sqlite
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=test-project, project_home=/Users/jrc/develop/timelink-py/tests/timelink-home/projects/test-project, db_type=sqlite, db_name=issue53, kleio_image=kleio-server, kleio_version=12.6.575, postgres_image=postgres, postgres_version=latest)

## import a file with groups with no direct mapping

In [4]:
# NBVAL_IGNORE_OUTPUT

import logging
logging.basicConfig(level=logging.INFO)

path = "sources/reference_sources/cronologias"
tlnb.update_from_sources(path=path)
# get the import status
import_status = tlnb.get_import_status(path=path)
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    print(errors)

In [3]:
# NBVAL_IGNORE_OUTPUT

import logging
logging.basicConfig(level=logging.INFO)

path = "sources/reference_sources/devassas"
tlnb.update_from_sources(path=path)
# get the import status
import_status = tlnb.get_import_status(path=path)
# list those with import_error > 0 or error > 0
errors = import_status.query("import_errors > 0 or errors > 0")
if len(errors) > 0:
    print(errors)

In [32]:
from timelink.api.models import Entity

for group, model in Entity.group_models.items():
    print(f"{group:24} - {model}")

devassa                  - <class 'timelink.api.models.act.Act'>
acusa                    - <class 'timelink.api.models.pom_som_mapper.Acusacoes'>
ls                       - <class 'timelink.api.models.attribute.Attribute'>
caso                     - <class 'timelink.api.models.pom_som_mapper.Caso'>
class                    - <class 'timelink.api.models.pom_som_mapper.PomSomMapper'>
crono                    - <class 'timelink.api.models.pom_som_mapper.Evento'>
lugar                    - <class 'timelink.api.models.geoentity.Geoentity'>
testo                    - <class 'timelink.api.models.person.Person'>
pai                      - <class 'timelink.api.models.person.Person'>
acusado                  - <class 'timelink.api.models.person.Person'>
acusada                  - <class 'timelink.api.models.person.Person'>
referida                 - <class 'timelink.api.models.person.Person'>
referido                 - <class 'timelink.api.models.person.Person'>
testa                    - <clas

In [29]:

from sqlalchemy import select

eventos, lugares, pessoas = tlnb.db.get_model(["crono", "lugar", "person"])

stmt = (select(eventos, lugares)
        .join(lugares,
              eventos.id == lugares.inside)
        .order_by(lugares.name, eventos.the_date))

with tlnb.db.session() as session:
    results = session.execute(stmt).all()
    for evento, lugar in results:
        print(f"{lugar.name:16} {evento.the_date} {evento.description}")
        in_query = select(pessoas).where(pessoas.inside == evento.id)
        people_in_event = session.execute(in_query).scalars().all()
        for person in people_in_event:
            print(f"{' ' * 32} {person.groupname} {person.name}")

print()
print(stmt)


Macau            15551120 Chegada primeiros jesuítas a Macau
                                 referido Belchior Nunes Barreto
                                 referido Gaspar Vilela
                                 referido António Dias
                                 referido Fernão Mendes Pinto
                                 referido Belchior Dias
                                 referido Estevão de Góis
Macau            15620000 Embaixada para Pequim em Macau
                                 referido Diogo Pereira
                                 referido Guilherme Pereira
                                 referido Luís Fróis
                                 referido Giovanni Battista de Monte
                                 referido Pedro Quintero
Pequim           15620000 Embaixada para Pequim em Macau
                                 referido Diogo Pereira
                                 referido Guilherme Pereira
                                 referido Luís Fróis
         

## Devassas



In [15]:

from sqlalchemy import select
from sqlalchemy import inspect

casos = tlnb.db.get_model("caso")
acusacoes = tlnb.db.get_model("acusa")

acusados = tlnb.db.get_model("acusado")  # acusado does not works

insp = inspect(casos)
print(insp.columns.keys())

insp = inspect(acusacoes)
print(insp.columns.keys())

insp = inspect(acusados)
print(insp.columns.keys())

['id', 'pom_class', 'inside', 'the_order', 'the_level', 'the_line', 'groupname', 'updated', 'indexed', 'name', 'the_type', 'obs', 'id', 'obs', 'the_type']
['id', 'pom_class', 'inside', 'the_order', 'the_level', 'the_line', 'groupname', 'updated', 'indexed', 'name', 'the_type', 'obs', 'id', 'idcaso', 'literal', 'obs', 'origem']
['id', 'pom_class', 'inside', 'the_order', 'the_level', 'the_line', 'groupname', 'updated', 'indexed', 'id', 'name', 'sex', 'obs']


In [16]:
casos, acusados = tlnb.db.get_model(["caso", "acusado"])

stmt = (select(casos, acusados)
        .join(acusados, acusados.inside == casos.id).order_by(casos.the_line))

with tlnb.db.session() as session:
    results = session.execute(stmt).all()
    caso_id = None
    for caso,acusados in results:
        if caso_id != caso.id:
            caso_id = caso.id
            print()
            print(caso.id, caso.obs)
        print(" ",acusados.name)


c1692-antonio-cordeiro-alcouc dava mulheres em casa. Margem: tirada de ambos 1.1.1691 extra_info: {"obs": {"original": "1o. 8.bro 691"}}
  antonio cordeiro
  francisca nunes

c1692-manuel-ferreira-juras pela hostia sagrada
  manuel ferreira

c1692-manuel-pinto-domingas-roxa portas adentro, ha 8 anos, tres filhos. Ja culpados anteriormente.
  manuel pinto
  domingas roxa

c1692-mateus-dias-curas benzedor de gado e de pessoas com grandes cerimonias no cimo de um monte, ha quatro anos atras. margem: tiradas
  mateus dias

c1692-catarina-gaspar-curas benze e cura de quebranto
  catarina gaspar

c1692-manuel-aires-domingas-luis ela seria prima da mulher dele
  manuel aires
  domingas luis

c1692-antonio-gomes-maria quarto grau margem: tirada dela em 25.1.692 dele 14.5.1693
  antonio gomes
  maria

c1692-jose-cunha-isabel-rodrigues ha mais de 6 anos,2 filhos
  jose da cunha
  isabel

c1692-antonio-catarina-fernandes tiveram filho(a). 2.o a t.a 23 manuel goncalves, ele foi prezo por ordem do

In [ ]:


stmt = (select(casos, acusacoes)
        .join(casos,
              acusacoes.idcaso == casos.id)
        .order_by(casos.the_line, acusacoes.the_line))

with tlnb.db.session() as session:
    results = session.execute(stmt).all()
    for caso, acusacao in results:
        print(caso.the_type,caso.obs)
        print(acusacao)
        # print(f"{lugar.name:16} {evento.the_date} {evento.description}")

print()
print(stmt)